## Imports e downloads

In [ ]:
!pip install pymupdf
!pip install faiss-cpu

from google.colab import drive

import requests
import fitz  # PyMuPDF
from io import BytesIO

import re

from sentence_transformers import SentenceTransformer

import faiss

import numpy as np

from sklearn.preprocessing import normalize

##carregamento pdf do drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PDF = "/content/drive/MyDrive/PDF INSTITUICAO.pdf"

## Criacao da funcao que vai extrair o texto do pdf PyMuPDF (fitz)


In [ ]:
def extrair_texto_pdf(PDF):
    doc = fitz.open(PDF)
    textos = [pagina.get_text() for pagina in doc]
    texto_completo = "\n".join(textos)
    return texto_completo, doc  # retorna o texto e o doc

## primeiros testes


In [ ]:
texto_completo, doc = extrair_texto_pdf(PDF)
# Ver quantas páginas tem
print(f"O PDF tem {len(doc)} páginas")

# Mostrar o conteúdo da primeira página
print("\nConteúdo da primeira página:\n")
print(doc[0].get_text())

O PDF tem 1 páginas

Conteúdo da primeira página:

1.​ "O ato de se matricular pode ser realizado diretamente na CRA, com ou sem 
procuração." 
 
2.​ "A renovação da matrícula é obrigatória para alunos em estágio 
supervisionado." 
 
3.​ "O projeto está em desenvolvimento e depende de aprovação do comitê." 
 
4.​ "A renovação do contrato de prestação de serviços deve ser feita até o fim do 
semestre." 
 
5.​ "Quem não renovar a matrícula será automaticamente desligado do curso." 
 
6.​ "Na modalidade EAD, a inscrição é feita por módulos, conforme regras 
específicas da instituição." 
 
7.​ "Estudantes presenciais devem se inscrever em disciplinas, respeitando o 
plano pedagógico do curso." 
 



##criacao dos chunks e testes

In [ ]:
def dividir_chunks_por_aspas(texto):
    #Captura todo conteúdo entre aspas duplas (inclusive quebra de linha)
    chunks = re.findall(r'"(.*?)"', texto, flags=re.DOTALL)
     #Remove espaços extras e ignora strings vazias
    return [c.strip() for c in chunks if c.strip()]

chunks = dividir_chunks_por_aspas(texto_completo)

In [ ]:
print(f"Total de chunks gerados: {len(chunks)}")

if len(chunks) == 0:
    print("⚠️ Nenhum chunk foi gerado. Verifique se o texto foi lido corretamente.")
else:
    for i in range(min(10, len(chunks))):
        print(f"🔹 \033[1m Chunk  {i+1}:\033[0m \n{chunks[i]}\n{'-'*80}")

Total de chunks gerados: 7
🔹  Chunk  1: 
O ato de se matricular pode ser realizado diretamente na CRA, com ou sem 
procuração.
--------------------------------------------------------------------------------
🔹  Chunk  2: 
A renovação da matrícula é obrigatória para alunos em estágio 
supervisionado.
--------------------------------------------------------------------------------
🔹  Chunk  3: 
O projeto está em desenvolvimento e depende de aprovação do comitê.
--------------------------------------------------------------------------------
🔹  Chunk  4: 
A renovação do contrato de prestação de serviços deve ser feita até o fim do 
semestre.
--------------------------------------------------------------------------------
🔹  Chunk  5: 
Quem não renovar a matrícula será automaticamente desligado do curso.
--------------------------------------------------------------------------------
🔹  Chunk  6: 
Na modalidade EAD, a inscrição é feita por módulos, conforme regras 
específicas da instituiç

## carregamento do modelo de embeddings como Sentence-BERT (SBERT). Ele converte tanto perguntas quanto textos em vetores numéricos que preservam similaridade semântica.

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
vetores = model.encode(chunks).astype("float32")
vetores = normalize(vetores, axis=1, norm='l2')

In [ ]:
index = faiss.IndexFlatIP(vetores.shape[1])
index.add(vetores)

In [ ]:
def recuperar_top_k(pergunta, k=1):
    vetor_pergunta = model.encode([pergunta]).astype("float32")
    vetor_pergunta = vetor_pergunta / np.linalg.norm(vetor_pergunta, axis=1, keepdims=True)  # normaliza
    distancias, indices = index.search(vetor_pergunta, k)
    return [(chunks[i], distancias[0][j]) for j, i in enumerate(indices[0])]

## primeiros testes modelo


In [ ]:
def imprimir_top_k_respostas(pergunta, k=1):
    vetor_pergunta = model.encode([pergunta]).astype("float32")
    vetor_pergunta = vetor_pergunta / np.linalg.norm(vetor_pergunta, axis=1, keepdims=True)
    distancias, indices = index.search(vetor_pergunta, k)

    print(f"\033[1m🟥 Pergunta:\033[0m {pergunta}\n")
    for i, (idx, dist) in enumerate(zip(indices[0], distancias[0])):
        print(f"\033[1m🔹 Top {i+1} (Similaridade: {dist:.4f}):\033[0m")
        print(f"{chunks[idx]}\n")

In [ ]:
perguntas = [
    "O que acontece se o aluno não renovar a matrícula?",
    "Onde posso renovar a matrícula?",
    "O estágio é obrigatório?",
    "O que diferencia um estudo EAD do estudante presencial?",
    "Quando deve ser feita a renovação de matrícula?"
]

In [ ]:
print('='*80,"\033[1m\nIndice de Similaridade:\033[0m\n\n"
                                             "\033[1mQuanto mais proximo de -1:\033[0m Totalmente contrario\n"
                                             "\033[1mQuanto mais proximo de 0:\033[0m Sem Relacao\n"
                                             "\033[1mQuanto mais proximo de 1:\033[0m Similaridade total\n\n",'='*80)
#for pergunta in perguntas:
    #imprimir_top_k_respostas(pergunta, k=1)
    #print("\n")  # separação entre as perguntas

imprimir_top_k_respostas(perguntas[0], k=1)
print("\n",('-'*80))
imprimir_top_k_respostas(perguntas[1], k=3)
print("\n",('-'*80))
imprimir_top_k_respostas(perguntas[2], k=6)
print("\n",('-'*80))
imprimir_top_k_respostas(perguntas[3], k=1)
print("\n",('-'*80))
imprimir_top_k_respostas(perguntas[4], k=2)

Indice de Similaridade:

Quanto mais proximo de -1: Totalmente contrario
Quanto mais proximo de 0: Sem Relacao
Quanto mais proximo de 1: Similaridade total

🟥 Pergunta: O que acontece se o aluno não renovar a matrícula?

🔹 Top 1 (Similaridade: 0.7677):
Quem não renovar a matrícula será automaticamente desligado do curso.


 --------------------------------------------------------------------------------
🟥 Pergunta: Onde posso renovar a matrícula?

🔹 Top 1 (Similaridade: 0.7299):
Quem não renovar a matrícula será automaticamente desligado do curso.

🔹 Top 2 (Similaridade: 0.5186):
A renovação da matrícula é obrigatória para alunos em estágio 
supervisionado.

🔹 Top 3 (Similaridade: 0.4214):
O ato de se matricular pode ser realizado diretamente na CRA, com ou sem 
procuração.


 --------------------------------------------------------------------------------
🟥 Pergunta: O estágio é obrigatório?

🔹 Top 1 (Similaridade: 0.5962):
A renovação da matrícula é obrigatória para alunos em estágio

## teste com outro pdf maior para teste


In [ ]:
PDF = "/content/drive/MyDrive/PDF INSTITUCIONAL 2.pdf"

In [ ]:
texto_completo, doc = extrair_texto_pdf(PDF)
# Ver quantas páginas tem
print(f"O PDF tem {len(doc)} páginas")

# Mostrar o conteúdo da primeira página
print("\nConteúdo da primeira página:\n")
print(doc[0].get_text())

O PDF tem 2 páginas

Conteúdo da primeira página:

​
O modelo Salesforce/blip-vqa-base foi escolhido por se 
apresentar como uma solução eficiente, leve e de alto 
desempenho para tarefas de Visual Question Answering 
(VQA) — ou seja, ele é capaz de interpretar informações 
contidas em imagens e fornecer respostas em linguagem 
natural a partir dessas informações. Essa capacidade é 
fundamental em aplicações que exigem compreensão visual 
aliada ao processamento textual, como assistentes 
inteligentes, sistemas de acessibilidade e ferramentas de 
automação de análise de imagens. Multimodalidade 
Avançada O BLIP (Bootstrapped Language-Image 
Pretraining) representa uma arquitetura multimodal moderna, 
projetada especificamente para integrar as áreas de visão 
computacional e processamento de linguagem natural. Esse 
modelo foi treinado utilizando grandes volumes de dados que 
combinam imagens e textos, o que o torna capaz de 
compreender o conteúdo visual em conjunto com o contexto 
tex

In [ ]:
def dividir_chunks_por_ponto_final(texto):
    # Divide o texto por ponto final seguido de espaço ou fim de linha
    chunks = re.split(r'\.\s*', texto)
    return [c.strip() for c in chunks if c.strip()]

In [ ]:
chunks = dividir_chunks_por_ponto_final(texto_completo)

In [ ]:
print(f"Total de chunks gerados: {len(chunks)}")

if len(chunks) == 0:
    print("⚠️ Nenhum chunk foi gerado. Verifique se o texto foi lido corretamente.")
else:
    for i in range(min(15, len(chunks))):
        print(f"🔹 \033[1m Chunk  {i+1}:\033[0m \n{chunks[i]}\n{'-'*80}")

Total de chunks gerados: 15
🔹  Chunk  1: 
​
O modelo Salesforce/blip-vqa-base foi escolhido por se 
apresentar como uma solução eficiente, leve e de alto 
desempenho para tarefas de Visual Question Answering 
(VQA) — ou seja, ele é capaz de interpretar informações 
contidas em imagens e fornecer respostas em linguagem 
natural a partir dessas informações
--------------------------------------------------------------------------------
🔹  Chunk  2: 
Essa capacidade é 
fundamental em aplicações que exigem compreensão visual 
aliada ao processamento textual, como assistentes 
inteligentes, sistemas de acessibilidade e ferramentas de 
automação de análise de imagens
--------------------------------------------------------------------------------
🔹  Chunk  3: 
Multimodalidade 
Avançada O BLIP (Bootstrapped Language-Image 
Pretraining) representa uma arquitetura multimodal moderna, 
projetada especificamente para integrar as áreas de visão 
computacional e processamento de linguagem natural
-

In [ ]:
perguntas = [
    "O que significa o termo multimodal no BLIP?",
    "O blip-vqa-base é a versão compacta de qual família?",
    "Por que o modelo Salesforce/blip-vqa-base foi escolhido?",
    "Qual modelo de grande porte foi citado no texto como exemplo de alternativa mais robusta, porém mais pesada?",
]

In [ ]:
print('='*80,"\033[1m\nIndice de Similaridade:\033[0m\n\n"
                                             "\033[1mQuanto mais proximo de -1:\033[0m Totalmente contrario\n"
                                             "\033[1mQuanto mais proximo de 0:\033[0m Sem Relacao\n"
                                             "\033[1mQuanto mais proximo de 1:\033[0m Similaridade total\n\n",'='*80)
#for pergunta in perguntas:
   # imprimir_top_k_respostas(pergunta, k=1)
   # print("\n")  # separação entre as perguntas

imprimir_top_k_respostas(perguntas[0], k=1)
print("\n",('-'*80))
imprimir_top_k_respostas(perguntas[1], k=1)
print("\n",('-'*80))
imprimir_top_k_respostas(perguntas[2], k=1)
print("\n",('-'*80))
imprimir_top_k_respostas(perguntas[3], k=1)

Indice de Similaridade:

Quanto mais proximo de -1: Totalmente contrario
Quanto mais proximo de 0: Sem Relacao
Quanto mais proximo de 1: Similaridade total

🟥 Pergunta: O que significa o termo multimodal no BLIP?

🔹 Top 1 (Similaridade: 0.4598):
Multimodalidade 
Avançada O BLIP (Bootstrapped Language-Image 
Pretraining) representa uma arquitetura multimodal moderna, 
projetada especificamente para integrar as áreas de visão 
computacional e processamento de linguagem natural


 --------------------------------------------------------------------------------
🟥 Pergunta: O blip-vqa-base é a versão compacta de qual família?

🔹 Top 1 (Similaridade: 0.4179):
​
 
​
Alto Desempenho com Custo Computacional Moderado O 
blip-vqa-base é a versão compacta da família BLIP


 --------------------------------------------------------------------------------
🟥 Pergunta: Por que o modelo Salesforce/blip-vqa-base foi escolhido?

🔹 Top 1 (Similaridade: 0.3993):
​
O modelo Salesforce/blip-vqa-base foi esco